In [2]:
# Create a cleaned training IDENTITY set 
import pandas as pd
df_transact_train = pd.read_csv('train_identity.csv')

In [3]:
# Keep only columns with <1%  of null and na vaules
low_null_columns = (df_transact_train.isnull().sum()/len(df_transact_train)*100) < 20

low_nan_columns = []
index = 0
print("columns with less than 1% nan")
for column in low_null_columns:
    if column == True:
        print(low_null_columns.index[index])
        low_nan_columns.append(low_null_columns.index[index])
    index += 1

columns with less than 1% nan
TransactionID
id_01
id_02
id_05
id_06
id_11
id_12
id_13
id_15
id_16
id_17
id_19
id_20
id_28
id_29
id_31
id_35
id_36
id_37
id_38
DeviceType
DeviceInfo


In [4]:
df_low_nans = df_transact_train.loc[:, low_nan_columns]

In [5]:
df_low_nans.shape

(144233, 22)

In [6]:
df_low_nans.dropna().shape

(97614, 22)

In [7]:
df_no_nans = df_low_nans.dropna()

In [12]:
# Create cleaned TRANSACTION training dataset
df_trans = pd.read_csv('train_transaction.csv')

In [15]:
df_trans_cut = df_trans[['isFraud', 'TransactionID']]
df_no_nans = pd.merge(df_no_nans,df_trans_cut,how='left', on='TransactionID')
df_no_nans[:5]

TransactionID  id_01     id_02  id_05  id_06  id_11     id_12  id_13  \
0        2987008   -5.0   98945.0    0.0   -5.0  100.0  NotFound   49.0   
1        2987010   -5.0  191631.0    0.0    0.0  100.0  NotFound   52.0   
2        2987017   -5.0   61141.0    3.0    0.0  100.0  NotFound   52.0   
3        2987040  -10.0  116098.0    0.0    0.0  100.0  NotFound   52.0   
4        2987048   -5.0  257037.0    0.0    0.0  100.0  NotFound   52.0   

   id_15     id_16  ...               id_31  id_35  id_36 id_37 id_38  \
0    New  NotFound  ...  mobile safari 11.0      T      F     F     T   
1  Found     Found  ...         chrome 62.0      F      F     T     T   
2  Found     Found  ...         chrome 62.0      T      F     T     T   
3  Found     Found  ...         chrome 62.0      F      F     T     T   
4    New  NotFound  ...         chrome 62.0      F      F     T     T   

  DeviceType  DeviceInfo isFraud_x isFraud_y isFraud  
0     mobile  iOS Device         0         0       0  
1    desktop     Windows         0         0       0  
2    desktop     Windows         0         0       0  
3    desktop     Windows         0         0       0  
4    desktop     Windows         0         0       0  

[5 rows x 25 columns]

In [16]:
df_no_nans.columns.values

array(['TransactionID', 'id_01', 'id_02', 'id_05', 'id_06', 'id_11',
       'id_12', 'id_13', 'id_15', 'id_16', 'id_17', 'id_19', 'id_20',
       'id_28', 'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo', 'isFraud_x', 'isFraud_y', 'isFraud'],
      dtype=object)

In [17]:
df_no_nans['isFraud'].value_counts()

0    89973
1     7641
Name: isFraud, dtype: int64

In [22]:
from random import sample
from sklearn import preprocessing

le = preprocessing.LabelEncoder() #Encode labels with value between 0 and n_classes-1

df_no_nans['id_15'] = le.fit_transform(df_no_nans['id_15'])
df_no_nans['id_16'] = le.fit_transform(df_no_nans['id_16'])
df_no_nans['id_12'] = le.fit_transform(df_no_nans['id_12'])
df_no_nans['id_13'] = le.fit_transform(df_no_nans['id_13'])
df_no_nans['id_17'] = le.fit_transform(df_no_nans['id_17'])
df_no_nans['id_19'] = le.fit_transform(df_no_nans['id_19'])
df_no_nans['id_20'] = le.fit_transform(df_no_nans['id_20'])
df_no_nans['id_28'] = le.fit_transform(df_no_nans['id_28'])
df_no_nans['id_29'] = le.fit_transform(df_no_nans['id_29'])
df_no_nans['id_31'] = le.fit_transform(df_no_nans['id_31'])
df_no_nans['id_35'] = le.fit_transform(df_no_nans['id_35'])
df_no_nans['id_36'] = le.fit_transform(df_no_nans['id_36'])
df_no_nans['id_37'] = le.fit_transform(df_no_nans['id_37'])
df_no_nans['id_38'] = le.fit_transform(df_no_nans['id_38'])
df_no_nans['DeviceType'] = le.fit_transform(df_no_nans['DeviceType'])
df_no_nans['DeviceInfo'] = le.fit_transform(df_no_nans['DeviceInfo'])

mask = df_no_nans['isFraud'] == 1

ident_RAW_Fraud = df_no_nans[mask] #length: 7641
ident_RAW_Non = df_no_nans[-mask]

ident_RAW_NonFraud = ident_RAW_Non.sample(n = 7641)

# Create balanced TRANSACTION training set with half known frauds and half known non-frauds
df_train_balanced = pd.concat([ident_RAW_NonFraud, ident_RAW_Fraud])

print (len(ident_RAW_Fraud))
print (len(ident_RAW_NonFraud))
print (len(df_train_balanced))

#for variable in cat_list:
#    le.fit(df_no_nans['{}'.format(variable)])
#    df_no_nans['{}'.format(variable)] = le.transform()
#le.fit(['chrome','chrome','safari','firefox'])
#le.transform

7641
7641
15282


In [28]:
df_train_balanced.to_csv('cleaned train_identity.csv') #TODO: df_train_balanced is from the TRANSACTION dataset. rename to cleaned_train_transaction?

In [29]:
df_no_nans.to_csv('identity_test.csv') #TODO: rename as cleaned_identity_test?